<a href="https://colab.research.google.com/github/MCostanzi/NumCounts/blob/master/red_sequence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Overview

This notebook demonstrates how to identify and model the galaxy red sequence using COSMOS, DES and SPT data. Explanatory text is added throughout to clarify each step.

### Installing dependencies

The first code cell installs the Python packages required for the analysis, including `emcee` for MCMC sampling and `pygtc` for plotting utilities.

In [ ]:
%pip install emcee pygtc
from astropy.table import Table  # Import Table for working with astronomical tables
import numpy as np
from matplotlib import pyplot as plt
import emcee
import pygtc
from tqdm import tqdm # Import tqdm for progress bars
from scipy.interpolate import interp1d
from astropy.stats import sigma_clip

In [ ]:
from google.colab import drive # Import drive to mount Google Drive
drive.mount('/content/drive') # mount google drive

### Loading COSMOS data

The COSMOS catalog is read from a FITS file stored on Google Drive. We inspect a few rows to ensure the data are loaded correctly.

In [ ]:
cosmos = Table.read("/content/drive/My Drive/ObservationalCosmology/Photo-z/cosmosM.fit")
cosmos[0:3]

### Computing rest-frame magnitudes and UVJ selection

COSMOS photometry is converted to U, V and J magnitudes. We apply a colour selection using the helper function `UVJsel` to flag passive galaxies.

In [ ]:
# Correct EZrest magnitudes to use the same definition detailed in Whitaker et al. (2011)
U = -2.5*np.log10(cosmos["EZrestU"]) + 23.9 # Calculate U magnitude
V = -2.5*np.log10(cosmos["EZrestV"]) + 23.9 # Calculate V magnitude
J = -2.5*np.log10(cosmos["EZrestJ"]) + 23.9 # Calculate J magnitude
cosmos["U"] = U # Add U magnitude to the table
cosmos["V"] = V # Add V magnitude to the table
cosmos["J"] = J # Add J magnitude to the table
z = cosmos["EZzphot"] # Get photometric redshift

# Define a cut based on redshift, source type, model fit, and stellar mass
cut = (cosmos["EZzphot"]<1.5) & (cosmos["SolModel"] != "PointSource") & (cosmos["FModel"] == 0) & (cosmos["loglpMassmed"] > 8.5)
#cut = np.isfinite(U) # Alternative cut for finite U magnitudes

def UVJsel(cosmos):
    """
    Selects passive galaxies based on UVJ colors and redshift.

    Args:
        cosmos (astropy.table.Table): The COSMOS catalog.
    """
    passive = np.zeros(len(cosmos),dtype=bool) # Initialize boolean array for passive selection
    U = -2.5*np.log10(cosmos["EZrestU"]) + 23.9 # Calculate U magnitude
    V = -2.5*np.log10(cosmos["EZrestV"]) + 23.9 # Calculate V magnitude
    J = -2.5*np.log10(cosmos["EZrestJ"]) + 23.9 # Calculate J magnitude
    z = cosmos["EZzphot"] # Get photometric redshift

    # Apply UVJ selection criteria for different redshift ranges
    passive[((U -V) > (0.88*(V - J) + 0.69)) & (0.0 < z) & (z < 0.5) & (U-V>1.3) & (V-J<1.6)] = True
    passive[((U -V) > 0.88*(V - J) + 0.59) & (0.5 <= z) & (z < 1.0) & (U-V>1.3) & (V-J<1.6)] = True
    passive[((U -V) > 0.88*(V - J) + 0.49) & (1.0 <= z) & (z < 2.0) & (U-V>1.3) & (V-J<1.6)] = True

    cosmos["passive"] = passive # Add passive selection to the table
#    return(passive) # Optionally return the boolean array
UVJsel(cosmos) # Apply the UVJ selection

### Visualising the UVJ diagram

A 2D histogram illustrates how galaxies populate the UVJ colour space. Selection boundaries for different redshift ranges are overplotted.

In [ ]:
def plotz05():
    """ Defines the UVJ selection boundary for redshift < 0.5. """
    x = np.linspace(-5,5,1000)
    y = np.zeros_like(x)
    y = (0.88*x + 0.69)
    y[y<1.3] = 1.3
    x[x>1.6] = 1.6
    return x,y

def plotz10():
    """ Defines the UVJ selection boundary for 0.5 < redshift < 1.0. """
    x = np.linspace(-5,5,1000)
    y = np.zeros_like(x)
    y = (0.88*x + 0.59)
    y[y<1.3] = 1.3
    x[x>1.6] = 1.6
    return x,y

def plotz20():
    """ Defines the UVJ selection boundary for 1.0 < redshift < 2.0. """
    x = np.linspace(-5,5,1000)
    y = np.zeros_like(x)
    y = (0.88*x + 0.49)
    y[y<1.3] = 1.3
    x[x>1.6] = 1.6
    return x,y

In [ ]:
UV = U[cut]-V[cut] # Calculate U-V color for selected galaxies
VJ = V[cut]-J[cut] # Calculate V-J color for selected galaxies
#plt.plot(VJ,UV,'.',alpha=0.1) # Plot all selected galaxies
#plt.plot(cosmos["V"][cosmos["passive"]]-cosmos["J"][cosmos["passive"]], cosmos["U"][cosmos["passive"]]-cosmos["V"][cosmos["passive"]],'.',alpha=0.01) # Plot passive galaxies

pcut = np.array(cosmos["passive"][cut]) # Get boolean array for passive selection within the cut
plt.figure(figsize=(8, 6)) # Set the figure size, optional
# Plot 2D histogram of all selected galaxies
plt.hist2d(VJ, UV, bins=50, cmap='ocean_r',range=[(0,2.3),(0,2.5)]) # Adjust the number of bins and colormap as needed
# Plot 2D histogram of passive galaxies
plt.hist2d(VJ[pcut], UV[pcut], bins=50, cmap='hot_r',range=[(0,2.3),(0,2.5)],alpha=0.45) # Adjust the number of bins and colormap as needed
#plt.plot(cosmos["V"][~cosmos["passive"]]-cosmos["J"][~cosmos["passive"]], cosmos["U"][~cosmos["passive"]]-cosmos["V"][~cosmos["passive"]],'g.',alpha=0.005) # Plot non-passive galaxies
#plt.plot(cosmos["V"][cosmos["passive"]]-cosmos["J"][cosmos["passive"]], cosmos["U"][cosmos["passive"]]-cosmos["V"][cosmos["passive"]],'r.',alpha=0.01) # Plot passive galaxies
#plt.colorbar() # Show the color bar representing the counts

x05,y05 = plotz05() # Get boundary for z < 0.5
plt.plot(x05,y05,'k:',label='z<0.5') # Plot boundary for z < 0.5
x10,y10 = plotz10() # Get boundary for 0.5 < z < 1.0
plt.plot(x10,y10,'k--',label='0.5<z<1') # Plot boundary for 0.5 < z < 1.0
x20,y20 = plotz20() # Get boundary for 1.0 < z < 2.0
plt.plot(x20,y20,'k',label='1<z<2') # Plot boundary for 1.0 < z < 2.0

plt.ylabel('UV') # Set x-axis label
plt.xlabel('VJ') # Set y-axis label
plt.title('2D Histogram of UV vs VJ') # Set title

plt.legend() # Show legend

### Filtering passive galaxies

After selecting passive objects we restrict the catalogue to only these galaxies for subsequent analysis.

In [ ]:
cosmos = cosmos[cut]

### DES filter definitions

Arrays containing the effective wavelengths of DES filters are defined. These are later used in the colour--magnitude relation model.

In [ ]:
filters = np.array([473, 642, 784, 926]) # Effective wavelengths of DES filters (g, r, i, z)
filters_b = np.array([398, 568, 710, 850]) # Blue edge wavelengths of DES filters
filters_r = np.array([548, 716, 857, 1002]) # Red edge wavelengths of DES filters
# Estimate the redshift at which the different bands are sensitive to the 4000A (400nm) break
print(filters_b/400-1) # Print normalized blue edge wavelengths
print(filters_r/400-1) # Print normalized red edge wavelengths

In [ ]:
MLAB = ["MAG_G","MAG_R","MAG_I","MAG_Z"] # Column names for DES magnitudes
MLABERR = ["MAGERR_G","MAGERR_R","MAGERR_I","MAGERR_Z"] # Column names for DES magnitude errors

### Loading DES photometry

DES data are read from disk and column names are standardised. Magnitude limits are applied to ensure quality photometry.

In [ ]:
des = Table.read("/content/drive/My Drive/ObservationalCosmology/Photo-z/DEScosmosy3deep.fits")
# Standardize column names for magnitudes and errors
des["MAG_G"] = des["BDF_MAG_G"]
des["MAG_R"] = des["BDF_MAG_R"]
des["MAG_I"] = des["BDF_MAG_I"]
des["MAG_Z"] = des["BDF_MAG_Z"]

des["MAGERR_G"] = des["BDF_MAG_ERR_DERED_G"]
des["MAGERR_R"] = des["BDF_MAG_ERR_DERED_R"]
des["MAGERR_I"] = des["BDF_MAG_ERR_DERED_I"]
des["MAGERR_Z"] = des["BDF_MAG_ERR_DERED_Z"]

mlim = [26.46,25.73,25.54,24.97] # Magnitude limits for DES filters (see: TableA1 arxiv:2012.12824)
blab = ["G","R","I","Z"] # Filter band names

# Apply magnitude limits to filter out low-quality photometry
for j,i in enumerate(blab):
    des = des[des["MAG_"+i]<mlim[j]]

# Apply additional cuts based on magnitude and error values
for ilab in range(3):
    test = (
    (des[MLAB[ilab]] > 15) &
    (des[MLAB[ilab + 1]] > 15) &
    (des[MLABERR[ilab]] > 0) &
    (des[MLABERR[ilab + 1]] > 0)
    )
    des = des[test]

### Cross-matching COSMOS with DES

Using `astropy` we match sources between COSMOS and DES within one arcsecond to obtain consistent colours.

In [ ]:
import astropy.units as u
from astropy.coordinates import SkyCoord

In [ ]:
c1 = SkyCoord(cosmos["RAJ2000"], cosmos["DEJ2000"]) # Create SkyCoord object for COSMOS sources
c2 = SkyCoord(des["RA"]*u.deg, des["DEC"]*u.deg) # Create SkyCoord object for DES sources

In [ ]:
max_sep = 1.0 * u.arcsec # Define maximum separation for matching
idx, d2d, d3d = c1.match_to_catalog_sky(c2) # Match COSMOS sources to DES sources
sep_constraint = d2d < max_sep # Create a constraint based on separation
cosmosm = cosmos[sep_constraint] # Filter COSMOS catalog based on separation constraint
desm = des[idx[sep_constraint]] # Filter DES catalog based on matching indices and separation constraint

In [ ]:
cosmosm[0:4], len(cosmosm) # Display the first 4 rows of the matched COSMOS table and its length

In [ ]:
desm[0:4] # Display the first 4 rows of the matched DES table

### Colour--magnitude diagnostic plots

These cells visualise galaxy colours versus magnitude and use sigma clipping to identify outliers.

In [ ]:
# Create a test sample based on redshift and passive selection
test = (cosmosm["EZzphot"]<0.5) & (cosmosm["EZzphot"]>0.45) & (cosmosm["passive"] )

In [ ]:
# Plot R-I color vs I magnitude for the test sample
plt.plot(desm["MAG_I"][test],desm["MAG_R"][test]-desm["MAG_I"][test],'.',alpha=0.3)
# Perform sigma clipping to identify outliers
sigmacl = sigma_clip(desm["MAG_R"][test]-desm["MAG_I"][test],sigma=3., cenfunc='median',stdfunc='mad_std',maxiters=None, masked=True)
sigmaclipped = sigmacl.mask # Get the mask indicating clipped values
# Plot the clipped outliers
plt.plot(desm["MAG_I"][test][sigmaclipped],desm["MAG_R"][test][sigmaclipped]-desm["MAG_I"][test][sigmaclipped],'.',alpha=0.3)
#plt.ylim(-1,3)

In [ ]:
# Plot R-I color vs I magnitude for the entire DES catalog
plt.plot(des["MAG_I"],des["MAG_R"]-des["MAG_I"],'.',alpha=0.3)
plt.ylim(-1,3) # Set y-axis limits
plt.xlim(15,25) # Set x-axis limits

In [ ]:
zrange = np.arange(0.05,1.5,0.075) # Define redshift bins
dz = zrange[1]-zrange[0] # Calculate redshift bin width

# Plot color-magnitude relation in different redshift bins
for i,j in enumerate(zrange):
    # Create a test sample for the current redshift bin and passive galaxies
    test = (cosmosm["EZzphot"]<j+dz) & (cosmosm["EZzphot"]>=j) & (cosmosm["passive"])
    # Plot R-I color vs I magnitude for the current redshift bin
    plt.plot(desm["MAG_I"][test][::10],desm["MAG_R"][test][::10]-desm["MAG_I"][test][::10],'.',alpha=1,label=str(j))
plt.legend() # Show legend
plt.xlim(15,25) # Set x-axis limits
plt.ylim(-1,2) # Set y-axis limits

### Fitting the colour--magnitude relation

In discrete redshift bins we fit a linear relation to the colours via likelihood maximisation. The resulting parameters are interpolated as smooth functions of redshift.

Note: To reduce the correlation between the fitted parameters, the x-axes of the linear relation is shifted by the median value of the sample:

$$ y = A + B \times (x- {\rm median}[x]) $$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import emcee
from scipy.stats import norm
from scipy.optimize import minimize


def CMRL(theta, x, y, xerr, yerr):
    """
    Log-likelihood function for fitting the color-magnitude relation with a mixture model.

    Args:
        theta (tuple): Parameters of the model (A, B, s, p, m0, ds).
        x (array): Magnitude values.
        y (array): Color values.
        xerr (array): Magnitude errors.
        yerr (array): Color errors.

    Returns:
        float: The log-likelihood value.
    """
    A, B, s,p,m0,ds = theta # Unpack parameters
    ymod = A + B * (x-np.median(x)) # Calculate the modeled color
    ymoderr2 = yerr**2 + s**2 + (xerr * B)**2 # Calculate the squared error of the modeled color
    # Calculate the log-likelihood using a mixture of a Gaussian (red sequence) and a uniform distribution (background)
    return np.sum(np.log(p*norm.pdf(y, loc=ymod, scale=np.sqrt(ymoderr2))+(1-p)*norm.pdf(y, loc=m0, scale=s+ds)))

def CMRL_clipped(theta, x, y, xerr, yerr):
    """
    Log-likelihood function for fitting the color-magnitude relation using sigma-clipped data.

    Args:
        theta (tuple): Parameters of the model (A, B, s).
        x (array): Magnitude values.
        y (array): Color values.
        xerr (array): Magnitude errors.
        yerr (array): Color errors.

    Returns:
        float: The log-likelihood value.
    """
    A, B, s = theta # Unpack parameters
    ymod = A + B * (x-np.median(x)) # Calculate the modeled color
    ymoderr2 = yerr**2 + s**2 + (xerr * B)**2 # Calculate the squared error of the modeled color
    # Calculate the log-likelihood using a Gaussian distribution
    return norm.logpdf(y, loc=ymod, scale=np.sqrt(ymoderr2)).sum()

def prior(theta):
    """
    Prior function for the mixture model parameters.

    Args:
        theta (tuple): Parameters of the model (A, B, s, p, m0, ds).

    Returns:
        float: The log-prior value (0 for valid parameters, -inf otherwise).
    """
    A, B, s,p,m0,ds = theta # Unpack parameters
    # Return 0 if parameters are within valid ranges, -inf otherwise
    return 0 if (s > 0) & (ds>0) & (p>0) & (p<1) else -np.inf

def prior_clipped(theta):
    """
    Prior function for the sigma-clipped model parameters.

    Args:
        theta (tuple): Parameters of the model (A, B, s).

    Returns:
        float: The log-prior value (0 for valid parameters, -inf otherwise).
    """
    A, B, s = theta # Unpack parameters
    # Return 0 if s is positive, -inf otherwise
    return 0 if (s > 0)  else -np.inf

def CMRp(theta, x, y, xerr, yerr):
    """
    Posterior function for the mixture model.

    Args:
        theta (tuple): Parameters of the model (A, B, s, p, m0, ds).
        x (array): Magnitude values.
        y (array): Color values.
        xerr (array): Magnitude errors.
        yerr (array): Color errors.

    Returns:
        float: The log-posterior value.
    """
    p = prior(theta) # Calculate the log-prior
    if p == 0:
        return CMRL(theta, x, y, xerr, yerr) + p # Return log-likelihood + log-prior
    else:
        return -np.inf # Return -inf if prior is -inf

def CMRp_clipped(theta, x, y, xerr, yerr):
    """
    Posterior function for the sigma-clipped model.

    Args:
        theta (tuple): Parameters of the model (A, B, s).
        x (array): Magnitude values.
        y (array): Color values.
        xerr (array): Magnitude errors.
        yerr (array): Color errors.

    Returns:
        float: The log-posterior value.
    """
    p = prior_clipped(theta) # Calculate the log-prior
    if p == 0:
        return CMRL_clipped(theta, x, y, xerr, yerr) + p # Return log-likelihood + log-prior
    else:
        return -np.inf # Return -inf if prior is -inf

In [ ]:
results = np.zeros((len(zrange), 4, 3))  # Initialize array to store fitting results (z bins, parameters+median, ilab)

# Loop over color indices (ilab)
for ilab in range(3):
    # Loop over redshift bins
    for i, j in enumerate(zrange):

        # Select galaxies in the current redshift bin and passive galaxies
        test1 = np.where((cosmosm["EZzphot"] < (j + dz)) & (cosmosm["EZzphot"] >= j) & (cosmosm["passive"]))[0]
        # Perform sigma clipping on the color values
        sigmacl = sigma_clip(desm[MLAB[ilab]][test1]-desm[MLAB[ilab+1]][test1],sigma=3., cenfunc='median',stdfunc='mad_std',maxiters=None, masked=True)
        sigmaclipped = sigmacl.mask # Get the mask indicating clipped values
        test = test1[sigmaclipped] # Apply the mask to the indices

        x = desm[MLAB[ilab + 1]][test] # Magnitude values (x-axis)
        y = desm[MLAB[ilab]][test] - desm[MLAB[ilab + 1]][test] # Color values (y-axis)
        xerr = desm[MLABERR[ilab + 1]][test] # Magnitude errors
        yerr = np.sqrt(desm[MLABERR[ilab]][test]**2 + desm[MLABERR[ilab + 1]][test]**2) # Color errors

        # Set initial guess for the optimization
        if i==0:
            p0 = np.random.rand(6)
            p0[2] = np.abs(p0[2])  # Ensure s is positive
            p0[5] = np.abs(p0[5])  # Ensure ds is positive
        else:
            p0=soln.x # Use the previous bin's solution as the initial guess

        np.random.seed(42) # Set random seed for reproducibility
        nll = lambda *args: -CMRp(*args) # Define the negative log-likelihood function
        initial = p0 # Set the initial guess
        soln = minimize(nll, initial, args=(x, y, xerr, yerr),method="Powell") # Perform optimization using Powell method
        results[i, 0:3, ilab] = soln.x[0:3] # Store the fitted parameters (A, B, s)
        results[i, 3, ilab] = np.median(x) # Store the median magnitude

# Creation of interpolation functions for each parameter and each ilab
interp_funcs = {}
for idx, param in enumerate(['A', 'B', 's','med']):
    interp_funcs[param] = [interp1d(zrange, results[:, idx, k], kind='quadratic', fill_value="extrapolate") for k in range(3)]

def CMR(z):
    """
    Returns the interpolated values of A, B, s, and med for each ilab given a redshift z.

    Args:
        z (float or array): Redshift value(s).

    Returns:
        list: A list containing interpolated values for A, B, s, and med for each ilab.
    """
    # Return interpolated values by evaluating each interpolation function at redshift z
    return [func(z) for sublist in interp_funcs.values() for func in sublist]



In [ ]:
CMR(0.46) # Evaluate the interpolated functions at redshift 0.46

In [ ]:
# (Plotting the results)
iilab = 2 # Select color index to plot
zz = np.linspace(0.05,1.5,1000) # Create a range of redshifts for plotting interpolation
plt.plot(zrange, results[:, 0,iilab], 'o',label='A') # Plot fitted A values
plt.plot(zz, CMR(zz)[iilab], label='A-') # Plot interpolated A values
plt.plot(zrange, results[:, 1,iilab], 'o',label='B') # Plot fitted B values
plt.plot(zz, CMR(zz)[iilab+3], label='B-') # Plot interpolated B values
plt.plot(zrange, results[:, 2,iilab], 'o',label='s') # Plot fitted s values
plt.plot(zz, CMR(zz)[iilab+6], label='s-') # Plot interpolated s values
plt.legend()
plt.show()


In [ ]:
def model(z,ilab):
    """
    Calculates the modeled color-magnitude relation for a given redshift and color index.

    Args:
        z (float): Redshift.
        ilab (int): Color index.

    Returns:
        tuple: x (magnitude values), ymod (modeled color values), s (scatter).
    """
    vec = CMR(z) # Get interpolated parameters at redshift z
    x = sptcat[MLAB[ilab+1]] # Magnitude values from SPT catalog
    y = sptcat[MLAB[ilab]] - sptcat[MLAB[ilab+1]] # Color values from SPT catalog
    A = vec[ilab] # Interporlated A parameter
    B = vec[ilab+3] # Interporlated B parameter
    s = vec[ilab+6] # Interporlated s parameter
    med = vec[ilab+9] # Interporlated median magnitude
    ymod = A + B * (x-med) # Calculate the modeled color
    return x,ymod,s # Return magnitude, modeled color, and scatter

zlim = [0.,0.37,0.79,1.5] # Redshift limits for different color indices
def like_p_of_z(theta,sptcat):
    """
    Log-likelihood function for fitting redshift and population fraction to SPT data.
    For the population fraction we use a mixture of a Gaussian (red sequence cluster members) and a Gaussian (background)

    Args:
        theta (tuple): Parameters (z=redshift, p= fraction of galay belonging to the cluster,
        [s0, s1, s2]= scatters of non-cluster members in the differen G-R, R-I, I-Z planes).
        sptcat (astropy.table.Table): The SPT catalog.

    Returns:
        float: The log-likelihood value.
    """
    z,p,s0,s1,s2 = theta # Unpack parameters (redshift, passive fraction, scatter parameters)
    logl = np.zeros(3) # Initialize array for log-likelihood of each color
    vec = CMR(z) # Get interpolated color-magnitude relation parameters at redshift z
    for ilab in [0,1,2]: # Loop over color indices
        x = sptcat[MLAB[ilab+1]] # Magnitude values
        y = sptcat[MLAB[ilab]] - sptcat[MLAB[ilab+1]] # Color values
        xerr = sptcat[MLABERR[ilab+1]] # Magnitude errors
        yerr = np.sqrt(sptcat[MLABERR[ilab]]**2+sptcat[MLABERR[ilab+1]]**2) # Color errors
        A = vec[ilab] # Interpolated A parameter
        B = vec[ilab+3] # Interpolated B parameter
        s = vec[ilab+6] # Interpolated scatter parameter
        med = vec[ilab+9] # Interpolated median magnitude
        ymod = A + B * (x-med) # Calculate the modeled color
        ymoderr2 = yerr**2 + s**2 + (xerr * B)**2 # Calculate the squared error of the modeled color
        sigma = theta[2+ilab] # Scatter parameter from input theta
        # Calculate log-likelihood using a mixture of a Gaussian (red sequence) and a Gaussian (background)
        logl[ilab]= np.sum(np.log(p*norm.pdf(y, loc=ymod, scale=np.sqrt(ymoderr2))+(1-p)*norm.pdf(y, loc=np.median(y), scale=sigma)))
    return np.sum(logl) # Return the total log-likelihood

def prior_p_of_z(theta):
    """
    Prior function for the SPT fitting parameters.

    Args:
        theta (tuple): Parameters (z, p, s0, s1, s2).

    Returns:
        float: The log-prior value (0 for valid parameters, -inf otherwise).
    """
    z,p,s0,s1,s2 = theta # Unpack parameters
    # Return 0 if parameters are within valid ranges, -inf otherwise
    if (z>0) & (z<1.2) & (p>0) & (p<1) & (s0>0) & (s1>0) & (s2>0):
        return 0
    else:
        return -np.inf

def post_p_of_z(theta,sptcat):
    """
    Posterior function for the SPT fitting.

    Args:
        theta (tuple): Parameters (z, p, s0, s1, s2).
        sptcat (astropy.table.Table): The SPT catalog.

    Returns:
        float: The log-posterior value.
    """
    lp = prior_p_of_z(theta) # Calculate the log-prior
    ll = like_p_of_z(theta,sptcat) # Calculate the log-likelihood
    if np.isfinite(lp+ll):
        return lp+ll
    else:
        return -np.inf

### Applying the model to SPT data

The interpolated relations are evaluated on SPT cluster galaxies and an MCMC sampler estimates the cluster properties.

In [ ]:
y6lim = [24.5,24,23.4,22.75] # Magnitude limits for SPT Y6 survey bands (g, r, i, z)
from astropy.table import Table # Import Table for working with astronomical tables
sptcat = Table.read("/content/drive/My Drive/ObservationalCosmology/Photo-z/sptclust.0.60.fits") # Read the SPT cluster catalog
# Apply selection cuts to the SPT catalog based on various flags and magnitude errors
sptcat = sptcat[(sptcat["FLAGS_FOOTPRINT"]==1) & (sptcat["FLAGS_GOLD"] == 0)& (sptcat["FLAGSTR"] == "ok") & (sptcat["FLAGS_FOREGROUND"] == 0) &
(sptcat["FITVD_FLAGS"] == 0) &  (sptcat["MASK_FLAGS"] == 0) & (np.abs(sptcat["SPREADERR_MODEL_G"]) <0.1) &
(sptcat["MAGERR_AUTO_G"] > 0) & (sptcat["MAGERR_AUTO_R"] > 0) & (sptcat["MAGERR_AUTO_I"] > 0) & (sptcat["MAGERR_AUTO_Z"] > 0)
& (sptcat["MAG_AUTO_G"] <y6lim[0]) & (sptcat["MAG_AUTO_R"] <y6lim[1]) & (sptcat["MAG_AUTO_I"] <y6lim[2]) & (sptcat["MAG_AUTO_Z"] <y6lim[3])]
#& (np.abs(sptcat["DEC"]-np.median(sptcat["DEC"]))<1./60) & # Optional cut based on median declination (commented out)
#(np.abs((sptcat["RA"]-np.median(sptcat["RA"]))*np.cos(np.deg2rad(np.median(sptcat["DEC"]))))<1./60)] # Optional cut based on median right ascension (commented out)
#sptcat # Display the filtered SPT catalog (commented out)

# Standardize column names for magnitudes and errors using BDF corrected values
sptcat["MAG_G"] = sptcat["BDF_MAG_G_CORRECTED"]
sptcat["MAG_R"] = sptcat["BDF_MAG_R_CORRECTED"]
sptcat["MAG_I"] = sptcat["BDF_MAG_I_CORRECTED"]
sptcat["MAG_Z"] = sptcat["BDF_MAG_Z_CORRECTED"]

# sptcat["MAG_G"] = sptcat["MAG_AUTO_G"] # Alternative: use AUTO magnitudes (commented out)
# sptcat["MAG_R"] = sptcat["MAG_AUTO_R"] # Alternative: use AUTO magnitudes (commented out)
# sptcat["MAG_I"] = sptcat["MAG_AUTO_I"] # Alternative: use AUTO magnitudes (commented out)
# sptcat["MAG_Z"] = sptcat["MAG_AUTO_Z"] # Alternative: use AUTO magnitudes (commented out)

# Standardize column names for magnitude errors using BDF errors
sptcat["MAGERR_G"] = sptcat["BDF_MAG_ERR_G"]
sptcat["MAGERR_R"] = sptcat["BDF_MAG_ERR_R"]
sptcat["MAGERR_I"] = sptcat["BDF_MAG_ERR_I"]
sptcat["MAGERR_Z"] = sptcat["BDF_MAG_ERR_Z"]
#MLAB = ["MAG_AUTO_G","MAG_AUTO_R","MAG_AUTO_I","MAG_AUTO_Z"] # Alternative magnitude column names (commented out)
#MLABERR = ["MAGERR_AUTO_G","MAGERR_AUTO_R","MAGERR_AUTO_I","MAGERR_AUTO_Z"] # Alternative magnitude error column names (commented out)
#MLABERR = ["BDF_MAG_ERR_G","BDF_MAG_ERR_R","BDF_MAG_ERR_I","BDF_MAG_ERR_Z"] # Alternative magnitude error column names (commented out)

# Calculate and print the log-posterior for different initial redshift guesses
print(post_p_of_z([0.25,0.9,1,1,1],sptcat),post_p_of_z([0.41,0.9,1,1,1],sptcat),post_p_of_z([0.93,0.9,1,1,1],sptcat))

In [ ]:
# Plot the G-band magnitude vs Right Ascension for the SPT catalog
plt.plot(sptcat['RA'],sptcat['MAG_G'],'.')

In [ ]:
nll = lambda *args: -post_p_of_z(*args) # Define the negative log-posterior function for optimization
#soln = minimize(nll,[0.4,0.9,1,1,1], args=(sptcat),method="Powell") # Initial optimization attempt (commented out)
#soln = minimize(nll,soln.x , args=(sptcat),method="Nelder-Mead") # Further optimization (commented out)
#soln = minimize(nll,[0.5,0.1,0.4,0.175,0.15] , args=(sptcat),method="Nelder-Mead") # Another initial guess and method (commented out)
initial = [0.1,0.1,0.4,0.175,0.15] # Define an initial guess for the parameters (z, p, s0, s1, s2)
bestf = np.inf # Initialize the best function value to infinity
bestp = initial # Initialize the best parameters to the initial guess
soln = minimize(nll,initial , args=(sptcat)) # Perform initial optimization
# Loop through different initial redshift values to find the best starting point
for i in np.arange(0.1,1.3,0.1):
    initial[0] = i # Set the initial redshift for the current iteration
    soln = minimize(nll,initial , args=(sptcat)) # Perform optimization with the current initial redshift
    if soln.fun < bestf: # Check if the current solution is better than the best found so far
        bestp = soln.x # Update the best parameters
        bestf = soln.fun # Update the best function value
soln = minimize(nll,bestp , args=(sptcat)) # Perform final optimization starting from the best found parameters
print(soln, post_p_of_z(soln.x,sptcat)) # Print the optimization result and the log-posterior at the best parameters

In [ ]:
# Create a figure with three subplots to visualize the color-magnitude relation for each color index
fig, axes = plt.subplots(1, 3, figsize=(15, 5), sharey=True)
for ilab in range(3): # Loop over color indices (0, 1, 2 for g-r, r-i, i-z)
    x = sptcat[MLAB[ilab + 1]] # Magnitude values for the current color index
    y = sptcat[MLAB[ilab]] - sptcat[MLAB[ilab + 1]] # Color values for the current color index
    xmod, ymod, s = model(soln.x[0], ilab) # Calculate the modeled color-magnitude relation using the fitted redshift

    ax = axes[ilab] # Select the current subplot
    ax.scatter(x, y, alpha=0.5, label=f'ilab={ilab}') # Plot the SPT galaxies as scatter points
    ax.plot(xmod, ymod, color='red', label='Model Fit') # Plot the modeled color-magnitude relation
    ax.set_title(f'Panel {ilab + 1} (ilab={ilab}) z={round(soln.x[0],2)}') # Set the title of the subplot
    ax.set_xlabel(MLAB[ilab + 1]) # Set the x-axis label
    ax.set_ylabel(f'{MLAB[ilab]} - {MLAB[ilab + 1]}') # Set the y-axis label
    ax.legend() # Show the legend

plt.tight_layout() # Adjust layout to prevent overlapping titles/labels
plt.show() # Display the plot

print(soln.x[0]) # Print the fitted redshift

In [ ]:
pos = soln.x * (1+1e-1 * np.random.randn(32, 5)) # Initialize the starting positions for the MCMC walkers with a small perturbation around the optimized parameters
#pos = [0.2,0.9,1] # Alternative initial positions (commented out)
nwalkers, ndim = pos.shape # Get the number of walkers and dimensions from the initial positions

sampler = emcee.EnsembleSampler(nwalkers, ndim, post_p_of_z, args=([sptcat])) # Initialize the MCMC sampler
sampler.run_mcmc(pos, 500, progress=True); # Run the MCMC sampler for 500 steps
fig, axes = plt.subplots(ndim, figsize=(10, 7), sharex=True) # Create a figure with subplots for each parameter to visualize the chains
samples = sampler.get_chain() # Get the MCMC chains
for i in range(ndim): # Loop over the parameters
    ax = axes[i] # Select the current subplot
    ax.plot(samples[:, :, i], "k", alpha=0.3) # Plot the chain for the current parameter
    ax.set_xlim(0, len(samples)) # Set the x-axis limits
    ax.yaxis.set_label_coords(-0.1, 0.5) # Adjust the position of the y-axis label

axes[-1].set_xlabel("step number"); # Set the x-axis label for the bottom subplot

In [ ]:
flat_samples = sampler.get_chain(flat=True,discard=200) # Get the flattened MCMC chains after discarding the burn-in period (first 200 steps)
plt.hist(flat_samples[:,0],bins=50) # Plot a histogram of the posterior distribution for the redshift (the first parameter

In [ ]:
zrange = np.arange(0.05,1.5,0.075) # Define redshift bins (re-defining for plotting purposes)
dz = zrange[1]-zrange[0] # Calculate redshift bin width (re-calculating)

#plt.legend() # Show legend (commented out)
for i,j in enumerate(zrange): # Loop over the redshift bins
    test = (cosmosm["EZzphot"]<j+dz) & (cosmosm["EZzphot"]>=j) & (cosmosm["passive"]) # Select passive galaxies within the current redshift bin
    fig, axes = plt.subplots(1, 3, figsize=(15, 5), sharey=True) # Create a figure with subplots for each color index
    for ilab in range(3): # Loop over color indices
        x = desm[MLAB[ilab + 1]][test] # Magnitude values
        y = desm[MLAB[ilab]][test] - desm[MLAB[ilab + 1]][test] # Color values
        xmod, ymod, s = model(np.mean(cosmosm["EZzphot"][test]), ilab) # Calculate the modeled color-magnitude relation using the mean redshift of the galaxies in the bin

        ax = axes[ilab] # Select the current subplot
        ax.scatter(x, y, alpha=0.5, label=f'ilab={ilab}') # Plot the DES galaxies in the current redshift bin
        ax.plot(xmod, ymod, color='red', label='Model Fit') # Plot the modeled color-magnitude relation
        ax.set_title(f'Panel {ilab + 1} (ilab={ilab}) z='+str(np.mean(cosmosm["EZzphot"][test]))) # Set the title of the subplot
        ax.set_xlabel(MLAB[ilab + 1]) # Set the x-axis label
        ax.set_ylabel(f'{MLAB[ilab]} - {MLAB[ilab + 1]}') # Set the y-axis label
        ax.legend() # Show the legend

    plt.tight_layout() # Adjust layout
    plt.show() # Display the plot

In [ ]:
filt = [(398,548),(568,716),(710,857),(850,1002)] # Define filter wavelength ranges
for i in filt: # Loop over the filter ranges
    print(i[0]/400.-1,i[1]/400.-1) # Print normalized filter ranges

In [ ]:
spt = Table.read("/content/drive/My Drive/ObservationalCosmology/Photo-z/SPTpol_500d_catalog_tablevOct3.fits")

In [ ]:
spt[spt["XI"]>10] # Display rows where the "XI" column is greater than 10

### Further analysis

The remaining cells load additional SPT catalogues and provide examples of exploratory plots and checks.